In [59]:
import pandas as pd
import numpy as np
import tushare as ts
pd.set_option('max_colwidth',200)

# 1. 常用函数基本信息介绍

=====fut_basic 输入参数=====

- exchange	str	Y	交易所代码 

- CFFEX-中金所 
- DCE-大商所 
- CZCE-郑商所 
- SHFE-上期所 
- INE-上海国际能源交易中心

fut_type	str	N	合约类型 

(1 普通合约 2主力与连续合约 默认取全部)

=====fut_basic 输出参数=====

- 名称	类型	默认显示	描述
- ts_code	str	Y	合约代码

- symbol	str	Y	交易标识
- exchange	str	Y	交易市场
- name	str	Y	中文简称
- fut_code	str	Y	合约产品代码
- multiplier	float	Y	合约乘数
- trade_unit	str	Y	交易计量单位
- per_unit	float	Y	交易单位(每手)
- quote_unit	str	Y	报价单位
- quote_unit_desc	str	Y	最小报价单位说明
- d_mode_desc	str	Y	交割方式说明
- list_date	str	Y	上市日期
- delist_date	str	Y	最后交易日期
- d_month	str	Y	交割月份
- last_ddate	str	Y	最后交割日
- trade_time_desc	str	N	交易时间说明

https://tushare.pro/document/2?doc_id=135

=====fut_settle 输入参数=====

名称	类型	必选	描述

- trade_date	str	N	交易日期 （trade_date/ts_code至少需要输入一个参数）
- ts_code	str	N	合约代码
- start_date	str	N	开始日期
- end_date	str	N	结束日期
- exchange	str	N	交易所代码

=====fut_settle 输出参数=====

名称	类型	默认显示	描述

- ts_code	str	Y	合约代码
- trade_date	str	Y	交易日期
- settle	float	Y	结算价
- trading_fee_rate	float	Y	交易手续费率
- trading_fee	float	Y	交易手续费
- delivery_fee	float	Y	交割手续费
- b_hedging_margin_rate	float	Y	买套保交易保证金率
- s_hedging_margin_rate	float	Y	卖套保交易保证金率
- long_margin_rate	float	Y	买投机交易保证金率
- short_margin_rate	float	Y	卖投机交易保证金率
- offset_today_fee	float	N	平今仓手续率
- exchange	str	N	交易所

https://tushare.pro/document/2?doc_id=141

# 2. 导入数据

In [50]:
all_exchanged = ["DCE", "CFFEX", "CZCE" ,"SHFE", "INE"]

In [54]:
pro = ts.pro_api("2f31c3932ead9fcc3830879132cc3ec8df3566550f711889d4a30f67")
df = []
for exchange in all_exchanged:
    # fut_type = 1 代表普通合约
    df_exchg = pro.fut_basic(exchange=exchange, fut_type='1', fields=
                             'ts_code,symbol,name,last_ddate,trade_time_desc,per_unit', 
                              last_ddate="Y", 
                              trade_time_desc="Y",
                              per_unit = "Y"
                              )
    df_exchg["exchange"] = exchange
    df.append(df_exchg)
df = pd.concat(df)

In [56]:
# fut_settle 需要600积分才可以调用：https://tushare.pro/document/2?doc_id=141

df = pro.fut_settle(trade_date='20181114', exchange='SHFE')

Exception: 抱歉，您没有权限访问该接口，权限的具体详情访问：https://tushare.pro/document/1?doc_id=108。

## 2.1 确定交易信号池

In [64]:
target_contracts = [
 "TA909", #1
 "SR1909", #2
 "OI909", #3
 "M1909", #4
 "AL1909", #5
 "ZN1909", #6
 "J1909", #7
 "BU1912", #8
 "RB1910", #9
 "CF909", #10
 "HC1910", #11
 "L1909", #12
 "RU2001", #13
 "ZC909", #14
 "P1909", #15
 "UR2001", #16
 "IF1909" #17
]

In [65]:
trgt_contract_df = df[df['symbol'].isin(target_contracts)]

In [66]:
trgt_contract_df

,ts_code,symbol,name,per_unit,last_ddate,trade_time_desc,exchange
999,J1909.DCE,J1909,焦炭1909,100.0,20190919,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:00(夜盘)",DCE
1001,L1909.DCE,L1909,塑料1909,5.0,20190919,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:00(夜盘)",DCE
1002,M1909.DCE,M1909,豆粕1909,10.0,20190919,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:00(夜盘)",DCE
1003,P1909.DCE,P1909,棕榈油1909,10.0,20190919,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:00(夜盘)",DCE
257,IF1909.CFX,IF1909,IF1909,1.0,20190920,"上午9:30-11:30,下午13:00-15:00",CFFEX
524,CF0909.ZCE,CF909,郑棉0909,5.0,20090916,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:30(夜盘)",CZCE
771,TA0909.ZCE,TA909,PTA0909,5.0,20090916,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:30(夜盘)",CZCE
866,ZC1909.ZCE,ZC909,动力煤1909,100.0,20190910,"上午9:00-11:30,下午13:30-15:00,以及交易所规定的其他交易时间,下午21:00-23:30(夜盘),最后交易日上午9:00-11:30",CZCE
870,CF1909.ZCE,CF909,郑棉1909,5.0,20190918,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:30(夜盘)",CZCE
871,TA1909.ZCE,TA909,PTA1909,5.0,20190918,"上午9:00-11:30,下午13:30-15:00,下午21:00-23:30(夜盘)",CZCE
